Custom Validators

In [ ]:
from pydantic import BaseModel, Field, ValidationError

class User(BaseModel):
    name: str
    age: int = Field(gt=21)
    
try:
    user = User(name="Alice", age=22)
    print(user)
except ValidationError as e:
    print(e)

name='Alice' age=22


In [4]:
from pydantic import field_validator

class User(BaseModel):
    name: str
    age: int = Field(gt=0)

    @field_validator("age")
    @classmethod
    def validate_age(cls, v):
        if v < 18 or v % 2 != 0:
            raise ValueError("Age must be even and at least 18.")
        return v

try:
    user = User(name="Alice", age=22)
    # would raise an exception because age is odd
    # user = User(name="Alice", age=21)
    print(user)
except ValueError as e:
    print(e)

name='Alice' age=22
